In [ ]:
from parse_documents import *

# filepath = 'upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'  # Replace with the path of your file
# filepath = "upload/World_Energy_By_Country_And_Region_1965_to_2023.csv"
filepath = 'upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
parse_document(filepath)



In [ ]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Print the type and keys of the parsed document
        print(type(parsed_document))
        print(parsed_document.keys())

        # Extract and print metadata if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            print("Metadata:")
            print(metadata)
        else:
            print("No metadata found.")
        
        # Print content if needed (commented to avoid large outputs)
        # print(parsed_document.get('content', 'No content available'))
        
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("path/to/your/document.pdf")


In [5]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Extract and print list of keys in the parsed document
        document_keys = list(parsed_document.keys())
        print("Keys in the parsed document:")
        print(document_keys)

        # Extract metadata keys if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            metadata_keys = list(metadata.keys())
            print("Keys in metadata:")
            print(metadata_keys)
        else:
            print("No metadata found.")

    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


Parsing successful: 200
Keys in the parsed document:
['metadata', 'content', 'status']
Keys in metadata:
['Number of Tables', 'File Modified Date', 'Compression Type', 'Data Precision', 'X-TIKA:Parsed-By-Full-Set', 'Number of Components', 'tiff:ImageLength', 'resourceName', 'Component 2', 'Component 1', 'Image Height', 'Image Width', 'File Size', 'Component 3', 'X-TIKA:Parsed-By', 'X-TIKA:parse_time_millis', 'X-TIKA:embedded_depth', 'File Name', 'Content-Length', 'tiff:BitsPerSample', 'tiff:ImageWidth', 'Content-Type']
